In [1]:
!pip install haversine tqdm --quiet           # 필요 라이브러리
import pandas as pd, numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import layers, models

In [2]:
SEQ_LEN     = 60      # 윈도우 길이(분) : 60 포인트
FEAT_COLS   = ['dlat', 'dlon', 'sog', 'cog', 'heading', 'dt']
LATENT_DIM  = 64
BATCH_SIZE  = 256
EPOCHS      = 50

In [3]:
PATH = '/content/AIS_2024_01_05.csv'
cols = ['MMSI','BaseDateTime','LAT','LON','SOG','COG','Heading']
df   = pd.read_csv(PATH, usecols=cols, parse_dates=['BaseDateTime'])
df.sort_values(['MMSI','BaseDateTime'], inplace=True)

In [4]:
# 선박별 시간 차이·위치 차이를 이용한 파생 변수
for col, src in [('dlat','LAT'), ('dlon','LON')]:
    df[col] = df.groupby('MMSI')[src].diff().fillna(0)
df['dt']     = df.groupby('MMSI')['BaseDateTime'].diff().dt.total_seconds().fillna(0)
df['sog']    = df['SOG'].fillna(0)
df['cog']    = df['COG'].fillna(0)
df['heading']= df['Heading'].fillna(0)

In [5]:
# ------------------------------------------------------------
# 3) 북한 선박(MMSI 445/447 시작) 제거 → 정상 패턴 학습용
# ------------------------------------------------------------
is_dprk = df['MMSI'].astype(str).str.startswith(('445','447'))
normal_df = df[~is_dprk].copy()

In [6]:
# ------------------------------------------------------------
# 4) 선박-ID 단위 Train/Val/Test 분리
# ------------------------------------------------------------
vessels        = normal_df['MMSI'].unique()
train_ids, tmp = train_test_split(vessels, test_size=0.30, random_state=42)  # 70%
val_ids, test_ids = train_test_split(tmp,   test_size=0.50, random_state=42) # 15% / 15%

split_map = {'train':train_ids, 'val':val_ids, 'test':test_ids}
def split_df(ids): return normal_df[normal_df['MMSI'].isin(ids)]

train_df, val_df, test_df = map(split_df, split_map.values())

print(f'Train vessels: {len(train_ids)}, rows: {len(train_df):,}')
print(f'Val   vessels: {len(val_ids)}, rows: {len(val_df):,}')
print(f'Test  vessels: {len(test_ids)}, rows: {len(test_df):,}')

Train vessels: 10377, rows: 5,186,187
Val   vessels: 2224, rows: 1,107,585
Test  vessels: 2224, rows: 1,103,262


In [7]:
# ------------------------------------------------------------
# 5) 시계열 윈도우 생성 함수
# ------------------------------------------------------------
def make_windows(df, seq_len=SEQ_LEN, feat_cols=FEAT_COLS):
    X = []
    for mmsi, g in tqdm(df.groupby('MMSI'), desc='Making windows'):
        arr = g[feat_cols].values
        for i in range(len(arr) - seq_len + 1):
            X.append(arr[i:i+seq_len])
    return np.stack(X, dtype=np.float32)

train_X = make_windows(train_df)
val_X   = make_windows(val_df)
test_X  = make_windows(test_df)

Making windows: 100%|██████████| 2224/2224 [00:01<00:00, 1859.83it/s]


In [8]:
# ------------------------------------------------------------
# 6) 정규화 (훈련 세트 기준)
# ------------------------------------------------------------
scaler = StandardScaler().fit(train_X.reshape(-1, len(FEAT_COLS)))
def scale(x): return scaler.transform(x.reshape(-1,len(FEAT_COLS))).reshape(x.shape)

train_X = scale(train_X)
val_X   = scale(val_X)
test_X  = scale(test_X)

print('train windows:', train_X.shape, '  val:', val_X.shape, '  test:', test_X.shape)

train windows: (4633337, 60, 6)   val: (988842, 60, 6)   test: (985371, 60, 6)


In [9]:
# ------------------------------------------------------------
# 7) LSTM-AutoEncoder
# ------------------------------------------------------------
inputs = layers.Input(shape=(SEQ_LEN, len(FEAT_COLS)))
x      = layers.LSTM(128, return_sequences=True)(inputs)
x      = layers.LSTM(LATENT_DIM, return_sequences=False)(x)
x      = layers.RepeatVector(SEQ_LEN)(x)
x      = layers.LSTM(LATENT_DIM, return_sequences=True)(x)
x      = layers.LSTM(128, return_sequences=True)(x)
outputs= layers.TimeDistributed(layers.Dense(len(FEAT_COLS)))(x)

model  = models.Model(inputs, outputs, name='LSTM_AE')
model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "LSTM_AE"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 60, 6)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 60, 128)        │        69,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ repeat_vector (RepeatVector)    │ (None, 60, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 60, 64)         │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 60, 128)        │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 60, 6)          │           774 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 251,142 (981.02 KB)

 Trainable params: 251,142 (981.02 KB)

 Non-trainable params: 0 (0.00 B)

In [10]:
history = model.fit(train_X, train_X,
                    epochs      = EPOCHS,
                    batch_size  = BATCH_SIZE,
                    validation_data = (val_X, val_X),
                    callbacks=[tf.keras.callbacks.EarlyStopping(
                        patience=5, restore_best_weights=True)])

Epoch 1/50
18099/18099 ━━━━━━━━━━━━━━━━━━━━ 381s 21ms/step - loss: 0.5579 - val_loss: 0.4777
Epoch 2/50
18099/18099 ━━━━━━━━━━━━━━━━━━━━ 368s 20ms/step - loss: 0.5228 - val_loss: 0.4758
Epoch 3/50
18099/18099 ━━━━━━━━━━━━━━━━━━━━ 374s 21ms/step - loss: 0.5162 - val_loss: 0.4695
Epoch 4/50
18099/18099 ━━━━━━━━━━━━━━━━━━━━ 376s 21ms/step - loss: 0.4786 - val_loss: 0.4619
Epoch 5/50
18099/18099 ━━━━━━━━━━━━━━━━━━━━ 374s 21ms/step - loss: 0.4734 - val_loss: 0.4558
Epoch 6/50
18099/18099 ━━━━━━━━━━━━━━━━━━━━ 373s 21ms/step - loss: 0.4875 - val_loss: 0.4632
Epoch 7/50
18099/18099 ━━━━━━━━━━━━━━━━━━━━ 376s 21ms/step - loss: 0.4714 - val_loss: 0.4574
Epoch 8/50
18099/18099 ━━━━━━━━━━━━━━━━━━━━ 375s 21ms/step - loss: 0.4635 - val_loss: 0.4450
Epoch 9/50
18099/18099 ━━━━━━━━━━━━━━━━━━━━ 374s 21ms/step - loss: 0.4804 - val_loss: 0.4407
Epoch 10/50
18099/18099 ━━━━━━━━━━━━━━━━━━━━ 376s 21ms/step - loss: 0.4631 - val_loss: 0.4286
Epoch 11/50
18099/18099 ━━━━━━━━━━━━━━━━━━━━ 377s 21ms/step - loss: 0

In [11]:
# ------------------------------------------------------------
# 임계값(Threshold) 산정
# ------------------------------------------------------------
recon_train = model.predict(train_X, batch_size=BATCH_SIZE)
mse_train   = np.mean(np.square(train_X - recon_train), axis=(1,2))
THRESHOLD   = np.percentile(mse_train, 95)
print(f"Reconstruction-error threshold (95%): {THRESHOLD:.6f}")

18099/18099 ━━━━━━━━━━━━━━━━━━━━ 131s 7ms/step
Reconstruction-error threshold (95%): 0.169043


In [12]:
# ------------------------------------------------------------
# 테스트 세트 평가
# ------------------------------------------------------------
recon_test = model.predict(test_X, batch_size=BATCH_SIZE)
mse_test   = np.mean(np.square(test_X - recon_test), axis=(1,2))
y_pred     = (mse_test > THRESHOLD).astype(int)     # 1 = 의심, 0 = 정상
print(f"Test windows: {len(test_X)},   Predicted anomaly 비율: {y_pred.mean()*100:.2f}%")

3850/3850 ━━━━━━━━━━━━━━━━━━━━ 28s 7ms/step
Test windows: 985371,   Predicted anomaly 비율: 4.85%
